# Exemplos

Nesse notebook vamos apresentar alguns exemplos do uso de Transformers para diferentes tipos de tarefas, utilizando a biblioteca _transformers_ da HuggingFace.

Instalando as bibliotecas necessárias:

In [1]:
!pip install transformers
!pip install torch

## Classificação

No primeiro exemplo, vamos fazer uma simples classificação de texto (mais especificamente uma análise de sentimentos) de um texto em português.

Vamos usar o módulo _pipeline_ que abstrai algumas etapas do processo de forma a tornar o código mais enxuto.

Alternativamente, também poderíamos usar os módulos _AutoTokenizer_ (para fazer a tokenização do texto, conforme o modelo escolhido) e _AutoModelForSequenceClassification_ (para realizar a tarefa de análise de sentimentos), para ter maior controle sobre essas etapas.

In [2]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline
import torch

Então selecionamos um modelo (pesquisar no hub da HuggingFace), e instaciamos o tokenizer para preparar os inputs e o modelo em si que fará a inferência.

In [3]:
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Nesse caso simples em que estamos usando o módulo _pipeline_, criamos o classificador self-packed com o modelo e o tokenizer.

In [4]:
classifier = pipeline(
    model=model_name,  # or model=model,
    # tokenizer=tokenizer,
    device=0
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Finalmente, passamos o texto para o classificador e obtemos o resultado.

In [5]:
text = "Que meetup chato em plena segunda, não aguento mais, estou deprimido."
# text = "Que meetup legal, estou muito animado para começar a usar transformers."

result = classifier(text)

print(result)

[[{'label': 'positive', 'score': 0.17427247762680054}, {'label': 'neutral', 'score': 0.3294520378112793}, {'label': 'negative', 'score': 0.49627548456192017}]]


## Question Answering

Nesse exemplo, o modelo foi treinado para a tarefa específica de responder perguntas dado um contexto.

In [6]:
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2", device=0)
question = "Onde eu trabalho?"
context = "Meu nome é Renato e eu trabalho na Enforce."
qa_model(question = question, context = context)

{'score': 0.8544053435325623, 'start': 34, 'end': 43, 'answer': ' Enforce.'}

## Tradução

Agora vamos usar um modelo de tradução de português para inglês. O modelo default para essa tarefa na biblioteca transformers é o T5 (https://huggingface.co/t5-base).

In [7]:
en_pt_translator = pipeline("translation_pt_to_en", "unicamp-dl/translation-pt-en-t5", device=0)
en_pt_translator("Já pode almoçar?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': 'Can you already have lunch?'}]

## Summarization

Esse tipo de tarefa consiste em resumir um conteúdo de texto.

In [8]:
model_summarization = "csebuetnlp/mT5_multilingual_XLSum"
summarizer = pipeline("summarization", model_summarization, device=0)
summarizer("A ginástica artística feminina do Brasil mostrou muita força neste domingo, no encerramento das eliminatórias das Olimpíadas, classificando-se a oito finais, um recorde histórico. Só Rebeca estará em cinco: no individual geral (em segundo), no salto (segundo), no solo (segundo) e na trave (terceiro). Em todos os casos, ficou logo atrás de Simone Biles. As duas foram eliminadas nas assimétricas, com notas semelhantes…")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[{'summary_text': 'Rebeca Pereira é a primeira atleta brasileira que venceu as Olimpíadas de Atenas.'}]

## Text generation

Finalmente, a geração de texto é o tipo de tarefa mais utilizado. Os modelos mais recentes estão ficando tão poderosos que, mesmo sendo treinados para a tarefa de text-generation, eles conseguem performar muito bem em outras tarefas sem a necessidade de fine-tuning específico.

In [9]:
generator = pipeline('text-generation', model = 'pierreguillou/gpt2-small-portuguese', device=0)
generator("Estava andando distraidamente quando escorreguei em")

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

[{'generated_text': 'Estava andando distraidamente quando escorreguei em direção à estrada.\n\nOs policiais levaram a carro de uma ambulância de combate. Na cena, ela foi baleada com nove tiros. Um deles atingiu a cabeça, perfurando um pulmão, mas não'}]